In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from timeit import default_timer as timer

In [2]:
# 1) Load Geodata of 204 IMIS stations (some only collect wind and temperature data, others also snow data):
def load_imis_stations():
    imis_df = pd.read_csv('Raw_Data/00_SLF_imis_stations.csv', sep=';', skiprows=0)
    return imis_df

imis_df = load_imis_stations()
print(imis_df.head())

   code         station_name        lon        lat  elevation country_code  \
0  DAV2           Bärentälli   9.819411  46.698887       2558           CH   
1  VSC1  Piz San Jon Dadaint  10.339132  46.753262       3092           CH   
2  MUT2               Mutten   9.017484  46.858757       2481           CH   
3  TUM2             Val Miez   9.021468  46.781067       2191           CH   
4  GLA2               Guppen   9.037582  46.996628       1632           CH   

  canton_code       type  
0          GR  SNOW_FLAT  
1          GR       WIND  
2          GL  SNOW_FLAT  
3          GR  SNOW_FLAT  
4          GL  SNOW_FLAT  


In [27]:
# 2) Load historical data of avalanche accidents (3'301 records since 1998):
def load_accidents():
    acc_df = pd.read_csv('assets/Raw_Data/01_SLF_hist_avalanche_accidents.csv', sep=';',skiprows=0)
    return acc_df

acc_df = load_accidents()
#print(acc_df.head())

In [28]:
# 3) Load historical IMIS weather data:
# (Dataset >100MB, too large for GitHub)
def load_hist_measurements():
    hist_measure_df = pd.read_csv('assets/Raw_Data/02_SLF_hist_daily_measurements.csv', sep=';',skiprows=0)
    # Alternatively download from external source:
    # import gdown
    #url = 'https://drive.google.com/uc?export=download&id=1rPALSGmKxSrNJIYYBlqYvbsSrCo1x7mz'
    # output = 'assets/02_SLF_hist_daily_measurements.csv'
    # gdown.download(url, output, quiet=False)
    # hist_measure_df = pd.read_csv('assets/02_SLF_hist_daily_measurements.csv', sep=';',skiprows=0)
    return hist_measure_df

hist_measure_df = load_hist_measurements()
#print(hist_measure_df.head())

In [29]:
# 4) Load historical IMIS snow data:
def load_hist_snow():
    hist_snow_df = pd.read_csv('assets/Raw_Data/03_SLF_hist_daily_snow.csv', sep=';',skiprows=0)
    return hist_snow_df

hist_snow_df = load_hist_snow()
#print(hist_snow_df.head())

In [30]:
start = timer()

# 5) K-NN Model to fill missing elevation data in acc_df based on 3 nearest neighbors:
def fill_missing_elevation_with_knn(acc_df, n_neighbors):
    # Split data into two sets:
    acc_with_elevation = acc_df.dropna(subset=['start_zone_elevation']) # Only rows with elevation data
    acc_without_elevation = acc_df[acc_df['start_zone_elevation'].isna()] # Only rows without elevation data

    # Fit K-NN model based on data with elevation dataset:
    knn = NearestNeighbors(n_neighbors=n_neighbors)
    knn.fit(acc_with_elevation[['start_zone_coordinates_latitude', 'start_zone_coordinates_longitude']])

    # Find nearest neighbors for dataset without elevation:
    distances, indices = knn.kneighbors(
        acc_without_elevation[['start_zone_coordinates_latitude', 'start_zone_coordinates_longitude']]
    )

    # Fill missing elevation data with mean elevation values:
    for i, idx in enumerate(acc_without_elevation.index):
        neighbor_indices = indices[i]
        mean_elevation = round(acc_with_elevation.iloc[neighbor_indices]['start_zone_elevation'].mean(), 2)
        acc_df.at[idx, 'start_zone_elevation'] = mean_elevation # Append data to original DataFrame

    return acc_df

acc_complete_df = fill_missing_elevation_with_knn(acc_df,3)
acc_complete_df.to_csv('assets/Raw_Data/01_SLF_hist_avalanche_complete.csv', index=False)
print(f"Runtime: {timer()-start:.2f} s") # Runtime: 0.22 s

Runtime: 0.22 s


In [32]:
# 6) Mapp weather and snow data to avalanche accident points:

start = timer()
# Calculation function to determine distance between two points:
def calculate_distance(lt1, ln1, lt2, ln2):
    R = 6373.0  
    lat1 = np.deg2rad(lt1)
    lon1 = np.deg2rad(ln1)
    lat2 = np.deg2rad(lt2)
    lon2 = np.deg2rad(ln2)
    
    dlon = lon2 - lon1 
    dlat = lat2 - lat1
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

# Function to map data:
def find_closest_weather_stations(row):
    radius = 10 # km
    date = row["date"]
    latitude = row["start_zone_coordinates_latitude"]
    longitude = row["start_zone_coordinates_longitude"]
    filtered = hist_measure_df[hist_measure_df['measure_date'] == date]
    result = filtered[calculate_distance(latitude, longitude, filtered['imis_latitude'].values, filtered['imis_longitude'].values) <= radius]
    row["air_temp_mean_stations"] = result['air_temp_day_mean'].mean()
    row["wind_speed_mean_stations"] = result['wind_speed_day_mean'].mean()
    row["wind_speed_max_stations"] = result['wind_speed_day_max'].mean()
    row["snow_surf_temp_mean_stations"] = result['snow_surf_temp_day_mean'].mean()
    row["snow_ground_temp_mean_stations"] = result['snow_ground_temp_day_mean'].mean()
    return row

acc_mapped_I_df = acc_complete_df.apply(find_closest_weather_stations, axis=1)

def find_closest_snow_stations(row):
    radius = 10 # km
    date = row["date"]
    latitude = row["start_zone_coordinates_latitude"]
    longitude = row["start_zone_coordinates_longitude"]
    filtered = hist_snow_df[hist_snow_df['measure_date'] == date]
    result = filtered[calculate_distance(latitude, longitude, filtered['imis_latitude'].values, filtered['imis_longitude'].values) <= radius]
    row["snow_height_mean_stations"] = result['snow_height_cm'].mean()
    row["new_snow_mean_stations"] = result['new_snow_cm'].mean()
    return row

acc_mapped_II_df = acc_mapped_I_df.apply(find_closest_snow_stations, axis=1)
acc_mapped_II_df.to_csv('assets/01_SLF_hist_statistical_avalanche_data.csv', index=False)
print(acc_mapped_II_df.head)
print(f"Runtime: {timer()-start:.2f} s") # Runtime: 843.62 s


<bound method NDFrame.head of             date canton   municipality  start_zone_coordinates_latitude  \
0     1998-01-04     GR       Langwies                        46.825069   
1     1998-01-04     UR      Andermatt                        46.617104   
2     1998-01-08     GR          Davos                        46.733421   
3     1998-01-09     GR          Davos                        46.783051   
4     1998-01-10     GR        Fideris                        46.852684   
...          ...    ...            ...                              ...   
3296  2023-05-15     VS        Zermatt                        45.940559   
3297  2023-05-17     BE  Lauterbrunnen                        46.534843   
3298  2023-05-19     VS        Blatten                        46.486446   
3299  2023-05-28     VS    Fieschertal                        46.561817   
3300  2023-06-02     VS         Naters                        46.456055   

      start_zone_coordinates_longitude  start_zone_elevation  number_

In [5]:
# 7) Grouping of alpine regions based on cantons:
acc_mapped_III_df = pd.read_csv('assets/01_SLF_hist_statistical_avalanche_data.csv', sep=',', skiprows=0)

# Grouping of alpine regions based on cantons:
eastern_alps = {"AI", "AR", "SG", "GR", "GL"}
western_alps = {"VD", "FR", "GE", "NE", "JU", "BE"}
central_alps = {"NW", "OW", "LU", "UR", "SZ", "SO"}
southern_alps = {"TI", "VS"}

def assign_region(canton):
    if canton in eastern_alps:
        return "Eastern Alps"
    elif canton in western_alps:
        return "Western Alps"
    elif canton in central_alps:
        return "Central Alps"
    elif canton in southern_alps:
        return "Southern Alps"
    else:
        return "NONE"

# Add new column to DataFrame acc_df:
acc_mapped_III_df['alpine_region'] = acc_mapped_III_df['canton'].apply(assign_region)
acc_mapped_III_df.to_csv('assets/01_SLF_hist_statistical_avalanche_data.csv', index=False)

print(acc_mapped_III_df.head)


<bound method NDFrame.head of             date canton   municipality  start_zone_coordinates_latitude  \
0     1998-01-04     GR       Langwies                        46.825069   
1     1998-01-04     UR      Andermatt                        46.617104   
2     1998-01-08     GR          Davos                        46.733421   
3     1998-01-09     GR          Davos                        46.783051   
4     1998-01-10     GR        Fideris                        46.852684   
...          ...    ...            ...                              ...   
3296  2023-05-15     VS        Zermatt                        45.940559   
3297  2023-05-17     BE  Lauterbrunnen                        46.534843   
3298  2023-05-19     VS        Blatten                        46.486446   
3299  2023-05-28     VS    Fieschertal                        46.561817   
3300  2023-06-02     VS         Naters                        46.456055   

      start_zone_coordinates_longitude  start_zone_elevation  number_